In [1]:
import os
import pandas as pd
from modules.data_pipeline import process_data, get_path, export_file

export_file.export_to_csv('April', 'igv')
# dict = process_data.read(99, 'igv')
# paths = get_path.get(99,'igv')

Now processing ---> ('ICA', 'ica0505', 'AT01Data.csv')
Now processing ---> ('ICA', 'ica0505', 'AT03Data.csv')
No info or empty file: ('ica0507', 'AT03Data.csv')
No info or empty file: ('ica0507', 'AT06Data.csv')
Now processing ---> ('ICA', 'ica0508', 'AT02Data.csv')
Now processing ---> ('ICA', 'ica0508', 'AT06Data.csv')
Now processing ---> ('ICA', 'ica0509', 'AT02Data.csv')
Now processing ---> ('ICA', 'ica0509', 'AT03Data.csv')
Now processing ---> ('ICA', 'ica0510', 'AT01Data.csv')
Now processing ---> ('ICA', 'ica0510', 'AT03Data.csv')
Now processing ---> ('ICA', 'ica0510-0511', 'AT01Data.csv')
Now processing ---> ('ICA', 'ica0510-0511', 'AT03Data.csv')
Now processing ---> ('ICA', 'ica0511', 'AT03Data.csv')
Now processing ---> ('ICA', 'ica20240420', 'AT01Data.csv')
Now processing ---> ('ICA', 'ica20240420', 'AT02Data.csv')
Now processing ---> ('ICA', 'ica20240420', 'AT05Data.csv')
An error occurred when getting ica map config: 'None'
An error occurred when getting ica map config: 'None

In [ ]:
not os.path.exists('c:\\Users\\westwell\\Documents\\.WorkDocuments\\github\\westwell_da\\data\\raw\\W99\\ck20240408\\V001_IGVData_KPI.csv')

In [ ]:
for project, project_p in paths.items():
    print('\n\n', project)
    for filename, path in project_p.items():
        print(filename)
        print(path)

In [ ]:
for project, project_dict in dict.items():
    print(project)
    for filename in project_dict.keys():
        print(filename)
        for df in project_dict[filename]:
            display(df.head(1))
        

In [ ]:
import pandas as pd

def __strip_str__(df=pd.DataFrame):
    """
    Strip leading and trailing whitespaces from column names and string columns.

    Parameters:
    DataFrame (pd.DataFrame): Input DataFrame. 

    Returns:
    pd.DataFrame: DataFrame with leading and trailing whitespaces stripped from column names and string columns.
    """
    df.columns = df.columns.str.strip()
    
    # Select columns of type 'object'
    obj_cols = df.select_dtypes(['object']).columns
    
    # Strip leading and trailing whitespaces from string columns
    df[obj_cols] = df[obj_cols].apply(lambda x: x.str.strip())

    return df

# p = "C:\\Users\westwell\Documents\.WorkDocuments\github\westwell_da\data\\raw\W99\TJ20240409-0410kpi\A023_IGVData_KPI.csv"
# p = "C:\\Users\westwell\Documents\.WorkDocuments\github\westwell_da\data\\raw\W99\\tpy20240409\\tpy06_IGVData.csv"
# p = "C:\\Users\westwell\Documents\.WorkDocuments\github\westwell_da\data\\raw\W99\\WH20240408\宝旺8号\TD002_IGVData_KPI.csv"
# p = "C:\\Users\westwell\Documents\.WorkDocuments\github\westwell_da\data\\raw\W99\TS20240411\\556_IGVData.csv"
p = "C:\\Users\westwell\Documents\.WorkDocuments\github\westwell_da\data\\raw\W99\dl_20240408\dlqt_2_IGVData_KPI.csv"

df = pd.read_csv(p)
df = __strip_str__(df)

display(df['vesselVisitID'].value_counts().index.tolist(), df.shape)

data = df.copy()
val_to_drop = ['None', '']
data = data[data['vesselVisitID'].isin(val_to_drop)==False]

display(data['vesselVisitID'].value_counts().index.tolist(), data.shape)

In [ ]:
p = 'c:\\Users\westwell\Documents\.WorkDocuments\github\westwell_da\data\processed\W99\Igv_W99_TJ.csv'
p = os.path.abspath(p)

df = pd.read_csv(p, encoding='utf-8-sig', index_col=0)
df.columns

In [ ]:
def lmd_get_current_task_tag(current_task=str, mission_type=str, location_ref=str):
    if (current_task != 'DSCH') & (current_task != 'LOAD'):
        if ('YARD' in location_ref and mission_type=='DELIVER') or ('QC' in location_ref and mission_type=='RECEIVE'):
            return 'DSCH'
        elif('QC' in location_ref and mission_type=='DELIVER') or ('YARD' in location_ref and mission_type=='RECEIVE'):
            return 'LOAD'      
    else:
        return current_task
    
df['current_task_tag'] = df.apply(
                    lambda x: lmd_get_current_task_tag(x['current_task'], x['mission_type'], x['location_ref']),
                    axis=1)


df.columns

# Test all weekly file

In [ ]:
import os
import pandas as pd
import numpy as np

def get_all_weekly_file():
    root = 'c:\\Users\westwell\Documents\.WorkDocuments\github\westwell_da\data\processed\W99'
    pathlist = []
    for item in os.listdir(root):
        p = os.path.join(root, item)
        if os.path.isfile(p):
            pathlist.append(p)
    return pathlist

def lmd_rtg_operation(location_ref=str, task_stage=str):
    if ('YARD' in location_ref) & (task_stage=='Waiting for Operation'):
        return 'Waiting for Operation'
    else:
        return np.NaN

li = get_all_weekly_file()

In [ ]:
for p in li:
    print(p)

    df = pd.read_csv(p, encoding='utf-8-sig', index_col=0)

    print(df.columns)
    print()

In [ ]:
df.columns


In [ ]:
df['task_stage'].value_counts().index.tolist()

In [ ]:
path = os.path.abspath('c:\\Users\westwell\Documents\.WorkDocuments\github\westwell_da\data\processed\W17\Igv_W17_TPY.csv')

d = pd.read_csv(path, encoding='utf-8-sig')
d.columns

In [ ]:
def lambda_power_usage(soc_diff):
    if soc_diff > 0 and soc_diff<=5: return soc_diff
    elif soc_diff<=0 and soc_diff>-5: return 0
    elif soc_diff >5: return 5
    elif soc_diff<=-5: return -1


for vehicle, d2 in d.groupby('vehicle_id'):
    for cycle, data in d2.groupby('Cycle Tag'):
        data['soc_diff'] = data['soc'] - data['soc'].shift(-1)
        data['power_usage'] = data['soc_diff'].apply(lambda x: lambda_power_usage(x))
        print( data['soc_diff'].value_counts().index.tolist(),
              data['power_usage'].value_counts().index.tolist())
        print()